<a href="https://colab.research.google.com/github/fatuunreal/nlp-fatu-andi/blob/main/deteksi_SMS_spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PEMPROSESAN BAHASA ALAMI BERBASIS TEKS (NLP) ~ A11.4617
## Oleh : FATU RAHMAT (A11.2022.1481) & ANDI LAKSONO (A11.2022.14839)
#Deteksi Spam

## Load Library

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [14]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## load dataset

In [15]:
data = pd.read_csv('spam.csv')
data.head()

,Kategori,Pesan
0,spam,Secara alami tak tertahankan identitas perusah...
1,spam,Fanny Gunslinger Perdagangan Saham adalah Merr...
2,spam,Rumah -rumah baru yang luar biasa menjadi muda...
3,spam,4 Permintaan Khusus Pencetakan Warna Informasi...
4,spam,"Jangan punya uang, dapatkan CD perangkat lunak..."


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2636 entries, 0 to 2635
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Kategori  2636 non-null   object
 1   Pesan     2636 non-null   object
dtypes: object(2)
memory usage: 41.3+ KB


In [17]:
data['Kategori'].unique()

array(['spam', 'ham'], dtype=object)

In [18]:
Kategori = {'spam':0, 'ham':1}

data['Kategori'] = data['Kategori'].map(Kategori)

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2636 entries, 0 to 2635
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Kategori  2636 non-null   int64 
 1   Pesan     2636 non-null   object
dtypes: int64(1), object(1)
memory usage: 41.3+ KB


In [20]:
data.rename(columns = {"Kategori": "label", "Pesan": "teks"}, inplace = True)

In [21]:
data.head()

,label,teks
0,0,Secara alami tak tertahankan identitas perusah...
1,0,Fanny Gunslinger Perdagangan Saham adalah Merr...
2,0,Rumah -rumah baru yang luar biasa menjadi muda...
3,0,4 Permintaan Khusus Pencetakan Warna Informasi...
4,0,"Jangan punya uang, dapatkan CD perangkat lunak..."


## text preprocessing

## case folding

In [22]:
import re

# membuat ungsi untuk case folding
def casefolding(text):
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'[-+]?[0-9]+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.strip()
    return text

In [23]:
# membandingkan sebelum dan sesudah case folding
raw_sample = data['teks'].iloc[100]
case_folding = casefolding(raw_sample)

print('raw data\t : ', raw_sample)
print('case folding\t : ', case_folding)

raw data	 :  Re: Pil yang Disetujui Dokter LGW Seorang pria yang diberkahi dengan palu 7 - 8 "lebih baik dilengkapi daripada seorang pria dengan palu 5 - 6". Apakah Anda lebih suka lebih dari cukup untuk menyelesaikan pekerjaan atau gagal. itu benar -benar terserah Anda. Metode kami dijamin akan meningkatkan ukuran Anda dengan 1 - 3 "datang ke sini dan lihat bagaimana - - - - email net ini disponsori oleh: ThinkGeek Selamat datang di Geek Heaven. http: / / thinkgeek. milis
case folding	 :  re pil yang disetujui dokter lgw seorang pria yang diberkahi dengan palu    lebih baik dilengkapi daripada seorang pria dengan palu    apakah anda lebih suka lebih dari cukup untuk menyelesaikan pekerjaan atau gagal itu benar benar terserah anda metode kami dijamin akan meningkatkan ukuran anda dengan    datang ke sini dan lihat bagaimana     email net ini disponsori oleh thinkgeek selamat datang di geek heaven http   thinkgeek milis


## word normalization

In [24]:
key_norm = pd.read_csv('key_norm.csv')

def text_normalize(text):
    text = ' '.join([key_norm[key_norm['singkat'] == word]['hasil'].values[0]
    if (key_norm['singkat'] == word).any()
    else word for word in text.split()
    ])

    text == str.lower(text)
    return text

In [25]:
# membandingkan before dan after word normalization

raw_data = data['teks'].iloc[100]
word_normal = text_normalize(case_folding)

print('raw data\t:', raw_data)
print('word normalize\t:', word_normal)

raw data	: Re: Pil yang Disetujui Dokter LGW Seorang pria yang diberkahi dengan palu 7 - 8 "lebih baik dilengkapi daripada seorang pria dengan palu 5 - 6". Apakah Anda lebih suka lebih dari cukup untuk menyelesaikan pekerjaan atau gagal. itu benar -benar terserah Anda. Metode kami dijamin akan meningkatkan ukuran Anda dengan 1 - 3 "datang ke sini dan lihat bagaimana - - - - email net ini disponsori oleh: ThinkGeek Selamat datang di Geek Heaven. http: / / thinkgeek. milis
word normalize	: re pil yang disetujui dokter lgw seorang pria yang diberkahi dengan palu lebih baik dilengkapi daripada seorang pria dengan palu apakah anda lebih suka lebih dari cukup untuk menyelesaikan pekerjaan atau gagal itu benar benar terserah anda metode kami dijamin akan meningkatkan ukuran anda dengan datang ke sini dan lihat bagaimana email net ini disponsori oleh thinkgeek selamat datang di geek heaven http thinkgeek milis


## filtering (stopword removal)

In [26]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

stopwords_ind = stopwords.words('indonesian')

In [27]:
len (stopwords_ind)

758

In [28]:
stopwords_ind

['ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'antar',
 'antara',
 'antaranya',
 'apa',
 'apaan',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'artinya',
 'asal',
 'asalkan',
 'atas',
 'atau',
 'ataukah',
 'ataupun',
 'awal',
 'awalnya',
 'bagai',
 'bagaikan',
 'bagaimana',
 'bagaimanakah',
 'bagaimanapun',
 'bagi',
 'bagian',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'baik',
 'bakal',
 'bakalan',
 'balik',
 'banyak',
 'bapak',
 'baru',
 'bawah',
 'beberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'bekerja',
 'belakang',
 'belakangan',
 'belum',
 'belumlah',
 'benar',
 'benarkah',
 'benarlah',
 'berada',
 'berakhir',
 'berakhirlah',
 'berakhirnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'berarti',
 'berawal',
 'berbagai',
 'berdatangan',
 'beri',
 'berikan',
 'berikut'

In [29]:
# membuat fungsi stopword removal

# menambahkan kata kedalam stopwords
more_stopword = ['tsel', 'gb', 'rb', 'btw']
stopwords_ind = stopwords_ind + more_stopword

def remove_stop_word(text):
    clean_words = []
    text = text.split()
    for word in text:
        if word not in stopwords_ind:
            clean_words.append(word)
    return " ".join(clean_words)

In [30]:
raw_sample = data['teks'].iloc[100]
case_folding = casefolding(raw_sample)
stopwords_removal = remove_stop_word(case_folding)

print('raw data \t\t:', raw_data)
print('case folding \t\t:', case_folding)
print('stopword removal \t:', stopwords_removal)

raw data 		: Re: Pil yang Disetujui Dokter LGW Seorang pria yang diberkahi dengan palu 7 - 8 "lebih baik dilengkapi daripada seorang pria dengan palu 5 - 6". Apakah Anda lebih suka lebih dari cukup untuk menyelesaikan pekerjaan atau gagal. itu benar -benar terserah Anda. Metode kami dijamin akan meningkatkan ukuran Anda dengan 1 - 3 "datang ke sini dan lihat bagaimana - - - - email net ini disponsori oleh: ThinkGeek Selamat datang di Geek Heaven. http: / / thinkgeek. milis
case folding 		: re pil yang disetujui dokter lgw seorang pria yang diberkahi dengan palu    lebih baik dilengkapi daripada seorang pria dengan palu    apakah anda lebih suka lebih dari cukup untuk menyelesaikan pekerjaan atau gagal itu benar benar terserah anda metode kami dijamin akan meningkatkan ukuran anda dengan    datang ke sini dan lihat bagaimana     email net ini disponsori oleh thinkgeek selamat datang di geek heaven http   thinkgeek milis
stopword removal 	: re pil disetujui dokter lgw pria diberkahi palu

In [31]:
!pip -q install sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.5 MB/s eta 0:00:00


In [32]:
# merubah kata menjadi kata dasar
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

# membuat fungsi untuk stemming bahasa indonesia
def stemming(text):
    text = stemmer.stem(text)
    return text

In [33]:
raw_sample = data['teks'].iloc[100]
case_folding = casefolding(raw_sample)
stopwords_removal = remove_stop_word(case_folding)
text_stemming = stemming(stopwords_removal)

print('raw data \t\t:', raw_sample)
print('case folding \t\t:', case_folding)
print('stopword removal \t:', stopwords_removal)
print('stemming \t\t:', text_stemming)

raw data 		: Re: Pil yang Disetujui Dokter LGW Seorang pria yang diberkahi dengan palu 7 - 8 "lebih baik dilengkapi daripada seorang pria dengan palu 5 - 6". Apakah Anda lebih suka lebih dari cukup untuk menyelesaikan pekerjaan atau gagal. itu benar -benar terserah Anda. Metode kami dijamin akan meningkatkan ukuran Anda dengan 1 - 3 "datang ke sini dan lihat bagaimana - - - - email net ini disponsori oleh: ThinkGeek Selamat datang di Geek Heaven. http: / / thinkgeek. milis
case folding 		: re pil yang disetujui dokter lgw seorang pria yang diberkahi dengan palu    lebih baik dilengkapi daripada seorang pria dengan palu    apakah anda lebih suka lebih dari cukup untuk menyelesaikan pekerjaan atau gagal itu benar benar terserah anda metode kami dijamin akan meningkatkan ukuran anda dengan    datang ke sini dan lihat bagaimana     email net ini disponsori oleh thinkgeek selamat datang di geek heaven http   thinkgeek milis
stopword removal 	: re pil disetujui dokter lgw pria diberkahi palu

In [34]:
# membuat fungsi untuk menggabungkan seluruh langkah preprocessing
def text_preprocessing_process(text):
    text = casefolding(text)
    text = text_normalize(text)
    text = remove_stop_word(text)
    text = stemming(text)
    return text

In [ ]:
%%time
data['clean_teks'] = data['teks'].apply(text_preprocessing_process)

In [ ]:
data

In [ ]:
data.to_csv('spam_cleaned.csv')

In [ ]:
x = data['clean_teks']
y = data['label']

In [ ]:
x

In [ ]:
y

## features extraction

In [ ]:
# save model
import pickle

#TF_IDF
from sklearn.feature_extraction.text import TfidfVectorizer

#unigram
vec_TF_IDF = TfidfVectorizer(ngram_range=(1,1))
vec_TF_IDF.fit(x)

x_tf_idf = vec_TF_IDF.transform(x)

pickle.dump(vec_TF_IDF.vocabulary_,open("feature_tf-idf.sav", "wb"))

In [ ]:
# menampilkan vocabulary TF-IDF
vec_TF_IDF.vocabulary_

In [ ]:
# melihat jumlah fitur
print(len(vec_TF_IDF.get_feature_names_out()))

In [ ]:
# melihat fitur apa saja dalam corpus
print(vec_TF_IDF.get_feature_names_out())

In [ ]:
x1 = vec_TF_IDF.transform(x).toarray()
data_tabular_tf_idf = pd.DataFrame(x1,columns=vec_TF_IDF.get_feature_names_out())
data_tabular_tf_idf

## features selection

In [ ]:
x_train = np.array(data_tabular_tf_idf)
y_train = np.array(y)

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
chi2_features = SelectKBest(chi2, k=3000)
x_kbest_features = chi2_features.fit_transform(x_train, y_train)

# untuk reduced features
print('Original Feature number', x_train.shape[1])
print('Reduced Feature number', x_kbest_features.shape[1])

In [ ]:
Data = pd.DataFrame(chi2_features.scores_,columns=['Nilai'])
Data

In [ ]:
# menampilkan data fitur beserta nilainya

feature = vec_TF_IDF.get_feature_names_out()
feature
Data['Fitur'] = feature
Data

In [ ]:
# mengurutkan fitur terbaik
Data.sort_values(by='Nilai', ascending=False)

In [ ]:
mask = chi2_features.get_support()
mask

In [ ]:
# menampilkan fitur yang terpilih berdasarkan mask atau nilai tertinggi yg  sudah diterapkan pada chi square
new_feature=[]
for bool, f in zip(mask, feature):
    if bool :
        new_feature.append(f)
    selected_feature=new_feature
selected_feature

In [ ]:
# membuat vocabulary baru berdasarkan fitur yang terseleksi

new_selected_feature = {}
for (k,v) in vec_TF_IDF.vocabulary_.items():
    if k in selected_feature:
        new_selected_feature[k] = v

new_selected_feature

In [ ]:
len(new_selected_feature)

In [ ]:
pickle.dump(new_selected_feature,open("new_selected_feature_tf-idf.sav", "wb"))

In [ ]:
# menampilkan fitur-fitur yang sudah diseleksi

data_selected_feature = pd.DataFrame(x_kbest_features, columns=selected_feature)
data_selected_feature

## Modeling

In [ ]:
selected_x = x_kbest_features
selected_x

In [ ]:
# import library
import random
from sklearn.model_selection import train_test_split

# import algorima naive bayes
from sklearn.naive_bayes import MultinomialNB

In [ ]:
x = selected_x
y = data.label

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [ ]:
print('Banyaknya X_train : ', len(x_train))
print('Banyaknya X_test : ', len(x_test))
print('Banyaknya Y_train : ', len(y_train))
print('Banyaknya Y_test : ', len(y_test))

In [ ]:
text_algorithm = MultinomialNB()

In [ ]:
model = text_algorithm.fit(x_train, y_train)

In [ ]:
# membuat model prediksi
data_input = ("fanny gunslinger dagang saham merrill muzo colza capai esmark ramble pepatah segovia kelompok coba kansas tanzania ya bunglon pakai lanjut libretto kencang kencang pustaka kencang deonat coba hall mcdougall ya hepburn einsteinian earmark anak pohon babi duane palfrey sederhana fleksibel huzzah pepperoni tidur")
data_input = text_preprocessing_process(data_input)

# load
tfidf = TfidfVectorizer

loaded_vec = TfidfVectorizer(decode_error="replace", vocabulary=set(pickle.load(open("new_selected_feature_tf-idf.sav", "rb"))))

hasil = model.predict(loaded_vec.fit_transform([data_input]))

if(hasil == 0):
    s = "SMS Spam"
else:
    s = "SMS Normal"

print("Hasil Prediksi : \n", s)

## Evaluasi Model

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

predicted = model.predict(x_test)

CM = confusion_matrix(y_test, predicted)

print(classification_report(y_test, predicted))

In [ ]:
# menyimpan model
pickle.dump(model,open("model_spam.sav", "wb"))